<a href="https://colab.research.google.com/github/comojin1994/Deep_Learning_Study/blob/master/3step_lecture/Vanila_Convolutional_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
tf.__version__

TensorFlow 2.x selected.


'2.1.0'

### Hyperparameter

In [0]:
EPOCHS = 10

### Model

In [0]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        super(ConvNet, self).__init__()
        conv2d = tf.keras.layers.Conv2D
        maxpool = tf.keras.layers.MaxPool2D
        self.sequence = list()
        self.sequence.append(conv2d(16, 3, 1, padding='same', activation='relu')) # 28x28x16
        self.sequence.append(conv2d(16, 3, 1, padding='same', activation='relu')) # 28x28x16
        self.sequence.append(maxpool(2)) # 14x14x16

        self.sequence.append(conv2d(32, 3, 1, padding='same', activation='relu')) # 14x14x32
        self.sequence.append(conv2d(32, 3, 1, padding='same', activation='relu')) # 14x14x32
        self.sequence.append(maxpool(2)) # 7x7x32

        self.sequence.append(conv2d(64, 3, 1, padding='same', activation='relu')) # 7x7x64
        self.sequence.append(conv2d(64, 3, 1, padding='same', activation='relu')) # 7x7x64
        
        self.sequence.append(tf.keras.layers.Flatten()) # 1568
        self.sequence.append(tf.keras.layers.Dense(2048, activation='relu'))
        self.sequence.append(tf.keras.layers.Dense(10, activation='softmax'))

    def call(self, x, training=False, mask=None):
        for layer in self.sequence:
            x = layer(x)
        return x

### Define Training, Test loop

In [0]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)
    loss = loss_object(labels, predictions)

    test_loss(loss)
    test_accuracy(labels, predictions)

### Load Data

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255., x_test / 255.

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

### Define model, loss, optimizer and metrics

In [0]:
# Create Model
model = ConvNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### Training

In [8]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(
          epoch + 1,
          train_loss.result(),
          train_accuracy.result() * 100,
          test_loss.result(),
          test_accuracy.result() * 100
    ))

Epoch 1, Loss: 0.10976364463567734, Accuracy: 96.6433334350586, Test Loss: 0.048773523420095444, Test Accuracy: 98.45999908447266
Epoch 2, Loss: 0.07689350098371506, Accuracy: 97.63833618164062, Test Loss: 0.04366962984204292, Test Accuracy: 98.63500213623047
Epoch 3, Loss: 0.06253419816493988, Accuracy: 98.08499908447266, Test Loss: 0.040633589029312134, Test Accuracy: 98.73999786376953
Epoch 4, Loss: 0.053558167070150375, Accuracy: 98.36458587646484, Test Loss: 0.03976321965456009, Test Accuracy: 98.75749969482422
Epoch 5, Loss: 0.0477285236120224, Accuracy: 98.53600311279297, Test Loss: 0.038772184401750565, Test Accuracy: 98.82599639892578
Epoch 6, Loss: 0.04318998381495476, Accuracy: 98.67333221435547, Test Loss: 0.0380406491458416, Test Accuracy: 98.86666870117188
Epoch 7, Loss: 0.03951335698366165, Accuracy: 98.78881072998047, Test Loss: 0.036775730550289154, Test Accuracy: 98.92713928222656
Epoch 8, Loss: 0.03685711324214935, Accuracy: 98.87354278564453, Test Loss: 0.0365581028

In [9]:
model.summary()

Model: "conv_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  160       
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  4640      
_________________________________________________________________
conv2d_3 (Conv2D)            multiple                  9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_4 (Conv2D)            multiple                  184